## Importing Libraries

In [1]:
import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from models import trial_model, alexnet_model

### Load Dataset

In [2]:
dataset = pd.read_csv(r"./facial_expressions/image_emotion.csv")

In [3]:
dataset

,image,emotion
0,facial-expressions_2868585k.jpg,surprise
1,facial-expressions_2868584k.jpg,disgust
2,facial-expressions_2868582k.jpg,fear
3,Aaron_Eckhart_0001.jpg,neutral
4,Aaron_Guiel_0001.jpg,happiness
...,...,...
13684,SharmilaTagore_80.jpg,HAPPINESS
13685,SharmilaTagore_81.jpg,HAPPINESS
13686,SharmilaTagore_82.jpg,HAPPINESS
13687,SharmilaTagore_83.jpg,HAPPINESS


In [4]:
dataset["emotion"].value_counts()

neutral      6717
happiness    5309
HAPPINESS     387
surprise      356
anger         227
DISGUST       195
NEUTRAL       151
SADNESS       144
sadness       124
ANGER          24
fear           13
disgust        13
SURPRISE       12
contempt        9
FEAR            8
Name: emotion, dtype: int64

In [5]:
#Replacing emotion in capital to small
dataset['emotion'] = dataset['emotion'].replace({'NEUTRAL':'neutral','HAPPINESS':'happiness','DISGUST':'disgust',
                                                 'SADNESS':'sadness','ANGER':'anger','SURPRISE':'surprise',
                                                 'FEAR':'fear'})

num_outputs = len(dataset["emotion"].value_counts()) #number of classes
dataset["emotion"].value_counts()


neutral      6868
happiness    5696
surprise      368
sadness       268
anger         251
disgust       208
fear           21
contempt        9
Name: emotion, dtype: int64

### Splitting into train test data

In [6]:
x_train, x_test, y_train, y_test = train_test_split(dataset['image'], dataset['emotion'])

train_data = pd.concat([x_train, y_train],axis = 1)  #training dataframe
test_data = pd.concat([x_test, y_test],axis = 1)  #test dataframe


### Using ImageDataGenerator 

In [7]:
datagen = ImageDataGenerator(rescale=1./255, validation_split = 0.2) # rescaling the pixel values between 0 to 1.
test_datagen = ImageDataGenerator(rescale=1./255) # rescaling the pixel values between 0 to 1.

#Access training images from directory marked to their class labels in dataframe
train_generator = datagen.flow_from_dataframe(dataframe = train_data,
                                              directory= "./facial_expressions/images",
                                              x_col="image", y_col = "emotion",
                                              target_size = (350,350),
                                              subset = "training", batch_size = 64)  

#Access validation images from directory marked to their class labels in dataframe
validation_generator = datagen.flow_from_dataframe(dataframe = train_data,
                                                   directory= "./facial_expressions/images",
                                                   x_col="image", y_col = "emotion",
                                                   target_size = (350,350),
                                                   subset= "validation",batch_size = 64)

#Access test images from directory marked to their class labels in dataframe
test_generator = test_datagen.flow_from_dataframe(dataframe = test_data,
                                                  directory= "./facial_expressions/images",
                                                  x_col="image", y_col = "emotion",
                                                  target_size = (350,350))

#traget size is chosen (350,350) because maximum images have this dimension.

Found 8213 validated image filenames belonging to 8 classes.
Found 2053 validated image filenames belonging to 8 classes.
Found 3423 validated image filenames belonging to 8 classes.


In [8]:
label_indices = train_generator.class_indices #get indices of each class in one hot encoding done during flow from dataframe
label_indices

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'happiness': 4,
 'neutral': 5,
 'sadness': 6,
 'surprise': 7}

# Alexnet Model

In [9]:
model = alexnet_model((350,350,3), num_outputs)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 85, 85, 48)        17472     
_________________________________________________________________
batch_normalization (BatchNo (None, 85, 85, 48)        192       
_________________________________________________________________
activation (Activation)      (None, 85, 85, 48)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 42, 42, 48)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 128)       153728    
_________________________________________________________________
batch_normalization_1 (Batch (None, 42, 42, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 42, 42, 128)       0

In [10]:
model.fit(train_generator,
          steps_per_epoch=100,
          epochs = 20,
          validation_data = validation_generator
         )

Epoch 1/20
100/100 [==============================] - 19s 188ms/step - loss: 2.1042 - accuracy: 0.3137 - val_loss: 4.7271 - val_accuracy: 0.0122
Epoch 2/20
100/100 [==============================] - 63s 625ms/step - loss: 1.7583 - accuracy: 0.5175 - val_loss: 1.1962 - val_accuracy: 0.5558
Epoch 3/20
100/100 [==============================] - 28s 282ms/step - loss: 1.4639 - accuracy: 0.6775 - val_loss: 1.4257 - val_accuracy: 0.5363
Epoch 4/20
100/100 [==============================] - 18s 183ms/step - loss: 1.3882 - accuracy: 0.6587 - val_loss: 1.3166 - val_accuracy: 0.6069
Epoch 5/20
100/100 [==============================] - 19s 189ms/step - loss: 1.3177 - accuracy: 0.6637 - val_loss: 1.4687 - val_accuracy: 0.5343
Epoch 6/20
100/100 [==============================] - 20s 199ms/step - loss: 1.2085 - accuracy: 0.7212 - val_loss: 1.1688 - val_accuracy: 0.6405
Epoch 7/20
100/100 [==============================] - 22s 217ms/step - loss: 1.0671 - accuracy: 0.7337 - val_loss: 1.5607 - val_ac

In [11]:
# get accuracy on test set
pred = model.evaluate(test_generator)
print("Accuracy on test set:", pred[1])

# save the trained weights
model.save_weights("alexnet_model.h5")

107/107 [==============================] - 66s 617ms/step - loss: 0.6537 - accuracy: 0.8008
Accuracy on test set: 0.8007595539093018
